In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/601'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">601</td>
<td><a href="/box-office-chart/daily/2011/07/08">Jul 8, 2011</a></td>
<td><b><a href="/movie/Zookeeper#tab=summary">Zookeeper</a></b></td>
<td class="data"> $80,000,000</td>
<td class="data"> $80,360,866</td>
<td class="data"> $170,805,525</td>
</tr>
<tr><td class="data">602</td>
<td><a href="/box-office-chart/daily/2008/12/12">Dec 12, 2008</a></td>
<td><b><a href="/movie/Day-the-Earth-Stood-Still-The#tab=summary">The Day the Earth Stood Still</a></b></td>
<td class="data"> $80,000,000</td>
<td class="data"> $79,366,978</td>
<td class="data"> $237,370,350</td>
</tr>
<tr><td class="data">603</td>
<td><a href="/box-office-chart/daily/1997/08/08">Aug 8, 1997</a></td>
<td><b><a href="/movie/Conspiracy-Theory#tab=summary">Conspiracy Theory</a></b></td>
<td class="data"> $80,000,000</td>
<td class="data"> $76,118

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Jul 8, 2011': ['/box-office-chart/daily/2011/07/08',
  '601',
  'Jul 8, 2011',
  'Zookeeper',
  '\xa0$80,000,000',
  '\xa0$80,360,866',
  '\xa0$170,805,525'],
 'Dec 12, 2008': ['/box-office-chart/daily/2008/12/12',
  '602',
  'Dec 12, 2008',
  'The Day the Earth Stood Still',
  '\xa0$80,000,000',
  '\xa0$79,366,978',
  '\xa0$237,370,350'],
 'Aug 8, 1997': ['/box-office-chart/daily/1997/08/08',
  '603',
  'Aug 8, 1997',
  'Conspiracy Theory',
  '\xa0$80,000,000',
  '\xa0$76,118,990',
  '\xa0$142,783,718'],
 'Jun 12, 1998': ['/box-office-chart/daily/1998/06/12',
  '604',
  'Jun 12, 1998',
  'Six Days, Seven Nights',
  '\xa0$80,000,000',
  '\xa0$74,339,294',
  '\xa0$164,800,000'],
 'May 24, 2002': ['/box-office-chart/daily/2002/05/24',
  '605',
  'May 24, 2002',
  'Spirit: Stallion of the Cimarron',
  '\xa0$80,000,000',
  '\xa0$73,215,310',
  '\xa0$106,515,310'],
 'Jun 8, 2001': ['/box-office-chart/daily/2001/06/08',
  '619',
  'Jun 8, 2001',
  'Evolution',
  '\xa0$80,000,000',
  '\xa0$

In [6]:
budget7 = pd.DataFrame(movies).T  #transpose
budget7.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget7.shape

(93, 7)

In [7]:
budget7['budget'] = budget7['budget'].str.replace('$', '')
budget7['budget'] = budget7['budget'].str.replace(',', '')
budget7['budget'] = budget7['budget'].astype(int)

In [8]:
budget7.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Jul 8, 2011",/box-office-chart/daily/2011/07/08,601,"Jul 8, 2011",Zookeeper,80000000,"$80,360,866","$170,805,525"
"Dec 12, 2008",/box-office-chart/daily/2008/12/12,602,"Dec 12, 2008",The Day the Earth Stood Still,80000000,"$79,366,978","$237,370,350"
"Aug 8, 1997",/box-office-chart/daily/1997/08/08,603,"Aug 8, 1997",Conspiracy Theory,80000000,"$76,118,990","$142,783,718"
"Jun 12, 1998",/box-office-chart/daily/1998/06/12,604,"Jun 12, 1998","Six Days, Seven Nights",80000000,"$74,339,294","$164,800,000"
"May 24, 2002",/box-office-chart/daily/2002/05/24,605,"May 24, 2002",Spirit: Stallion of the Cimarron,80000000,"$73,215,310","$106,515,310"


In [9]:
budget7.to_csv('budget7.csv')